In [1]:
import json
import pickle
import torch
import faiss
import numpy as np
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv('.env')

e:\work\Projects\Arogyam\chatbot-backend\backend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=os.getenv("GEMINI_API_KEY"))
workflow = StateGraph(state_schema=MessagesState)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an empathetic, professional, and insightful verbal therapy chatbot. Your primary goal is to support users with their emotional and psychological well-being. You listen actively, provide thoughtful responses, and guide users toward self-reflection and positive coping strategies."
            "You should:"
            "Use a warm, non-judgmental, and supportive tone."
            "Ask open-ended questions to encourage users to express their feelings."
            "Offer evidence-based techniques from cognitive behavioral therapy (CBT), mindfulness, and positive psychology."
            "Never diagnose or prescribe medication—always encourage seeking professional help when necessary."
            "Prioritize user safety: If a user expresses thoughts of self-harm or harm to others, gently encourage them to seek immediate help from a trusted person or professional."
            "Maintain professionalism, avoid making assumptions, and adapt your responses to the user's needs while fostering a safe and healing conversation."


        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def input_func(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", input_func)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [4]:
# medical_meadow_health_advise
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return [entry['Context'] + " " + entry['Response'] for entry in data]

In [5]:
# Chunk the Data
def chunk_data(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    chunks = splitter.split_text("\n".join(data))
    with open('data/chunks/json_chunked_data.pkl', 'wb') as f:
        pickle.dump(chunks, f)
    return chunks

# Generate Embeddings
def embed(chunks):
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=chunks
    )
    tensor = torch.tensor(result['embedding'], dtype=torch.float32)
    return tensor

# Store Embeddings in FAISS
def store_embeddings(tensor):
    embeddings = np.array([embedding.cpu().numpy() for embedding in tensor])
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    faiss.write_index(index, "data/vectors/therapy.faiss")
    return index

# Search for Relevant Chunks
def search_embeddings(query_embedding, chunks, k=3):
    query_embedding_array = np.expand_dims(np.array(query_embedding, dtype='float32'), axis=0)
    index = faiss.read_index("data/vectors/therapy.faiss")
    distances, indices = index.search(query_embedding_array, k)
    matching_chunks = [chunks[i] for i in indices[0]]
    return matching_chunks

# Generate Response
def generate_response(matching_chunks, query, app):
    context = "\n".join(matching_chunks)
    print("Context:\n", context)
    input_prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    input_message = [HumanMessage(content=input_prompt)]
    output = app.invoke({"messages": input_message}, config)
    return output["messages"][-1].content

# Analyze JSON Data
def analyze_json(file_path, query, app):
    data = load_json(file_path)
    print("file loaded")
    chunked = chunk_data(data)
    print("Chunked")
    embeddings = embed(chunked)
    print("Embedded")
    store_embeddings(embeddings)
    print("Embedding saved")
    # query_embedding = embed(query)
    # results = search_embeddings(query_embedding, chunked)
    # response = generate_response(results, query, app)
    # return response

In [6]:
# json_file = "data\combined_dataset.json"
# query = ""
# ans = analyze_json(json_file, query, app)

In [7]:
query = "I am having a hard time dealing with everything going all around me. Actually I am a college student and this is my placement season going on, I know that I am good but I am feeling this pressure which is making me feel less than everyone around. I, somtimes feel very suffocated and depressed."
query_embedding = embed(query)
chunked = pickle.load(open('data\chunks\json_chunked_data.pkl', 'rb'))
results = search_embeddings(query_embedding, chunked)
response = generate_response(results, query, app)

print(response)

Context:
 help for my stress because I was having crying spells from the stress of going to school and confronting my classmates. After that, I was fine for a couple of years. I entered my first year of college in a really tough program with high demands of expected work. I believe my depression re-booted
help for my stress because I was having crying spells from the stress of going to school and confronting my classmates. After that, I was fine for a couple of years. I entered my first year of college in a really tough program with high demands of expected work. I believe my depression re-booted
I feel like I am not at a good state of mind. I'm very unsettled in my soul. I'm not happy with myself or the decisions I make, which makes me not happy with anyone else. I feel like a failure most of my days. I don't feel like I'm good at anything anymore. I feel like less of a person. It seems
It sounds like you're carrying a heavy burden right now, and it's brave of you to reach out.  The p

In [8]:
query = "yes please"
query_embedding = embed(query)
chunked = pickle.load(open('data\chunks\json_chunked_data.pkl', 'rb'))
results = search_embeddings(query_embedding, chunked)
response = generate_response(results, query, app)

print(response)

Context:
 invited.
it again. This only works if both of you agree to that before something starts, though.
it again. This only works if both of you agree to that before something starts, though.
Okay, I can help you find some resources. To do so effectively, I need a little more information.  Could you please tell me your location (country or state/province is sufficient)?  Knowing your location will allow me to find relevant helplines, mental health organizations, and online resources specific to your area.

In the meantime, here are some general resources that may be helpful:

* **The Crisis Text Line:** Text HOME to 741741 from anywhere in the US, anytime, about any type of crisis.
* **The National Suicide Prevention Lifeline:**  988 (in the US)  This is a confidential resource available 24/7.

Providing your location will allow me to find even more specific and potentially more helpful resources for you.  Remember, reaching out is a sign of strength, and there are people who care a

In [9]:
query = "I am from India"
query_embedding = embed(query)
chunked = pickle.load(open('data\chunks\json_chunked_data.pkl', 'rb'))
results = search_embeddings(query_embedding, chunked)
response = generate_response(results, query, app)

print(response)

Context:
 I am an international student, and it is my first semester in graduate school in the United States. I faced a cultural shock, and I was so depressed when I arrived here. My counselor in the university was my savior. He helped me a great deal. Now I am going back to my country for a vacation. I was
I am an international student, and it is my first semester in graduate school in the United States. I faced a cultural shock, and I was so depressed when I arrived here. My counselor in the university was my savior. He helped me a great deal. Now I am going back to my country for a vacation. I was
I am an international student, and it is my first semester in graduate school in the United States. I faced a cultural shock, and I was so depressed when I arrived here. My counselor in the university was my savior. He helped me a great deal. Now I am going back to my country for a vacation. I was
That's wonderful to hear that your university counselor was so supportive and helpful during 

In [4]:
import time
import pickle
import numpy as np
import torch
import faiss
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted
from dotenv import load_dotenv
import os
load_dotenv()
genai.configure(api_key=os.environ['GEMINI_API_KEY'])

def embed(query):
    # Your embedding logic here
    query_embed_result = genai.embed_content(model="models/text-embedding-004", content=query)
    return torch.tensor(query_embed_result['embedding'], dtype=torch.float32)

def search_embeddings(query_embedding, chunked):
    query_embedding_array = np.expand_dims(np.array(query_embedding, dtype='float32'), axis=0)
    index = faiss.read_index(r"data\\vectors\\therapy.faiss")
    distances, indices = index.search(query_embedding_array, 3)
    return [chunked[i] for i in indices[0]]

def generate_response(results, query):
    context = "\n".join(results)
    system_prompt = "You are an empathetic, professional, and insightful verbal therapy chatbot. Your primary goal is to support users with their emotional and psychological well-being. You listen actively, provide thoughtful responses, and guide users toward self-reflection and positive coping strategies. You should: Use a warm, non-judgmental, and supportive tone. Ask open-ended questions to encourage users to express their feelings.Offer evidence-based techniques from cognitive behavioral therapy (CBT), mindfulness, and positive psychology. Never diagnose or prescribe medication—always encourage seeking professional help when necessary. Prioritize user safety: If a user expresses thoughts of self-harm or harm to others, gently encourage them to seek immediate help from a trusted person or professional.Maintain professionalism, avoid making assumptions, and adapt your responses to the user's needs while fostering a safe and healing conversation."
    input_prompt = f"{system_prompt}\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    llm = genai.GenerativeModel('gemini-1.5-flash')

    max_retries = 5
    for attempt in range(max_retries):
        try:
            response = llm.generate_content(input_prompt)
            return response.text
        except ResourceExhausted as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print("Max retries reached. Quota exceeded.")
                raise e

# Example usage
query = "I am having a hard time dealing with everything going all around me. Actually I am a college student and this is my placement season going on, I know that I am good but I am feeling this pressure which is making me feel less than everyone around. I, somtimes feel very suffocated and depressed."
query_embedding = embed(query)
chunked = pickle.load(open('data\\chunks\\json_chunked_data.pkl', 'rb'))
results = search_embeddings(query_embedding, chunked)
response = generate_response(results, query)

print(response)

It sounds like you're going through a really challenging time, and it takes courage to share that with me.  Placement season is incredibly stressful, and it's completely understandable to feel the pressure mounting.  You mentioned feeling less than everyone around you – can you tell me more about what specifically is contributing to that feeling? What thoughts or comparisons are you making?

It’s important to remember that your feelings of suffocation and depression are valid responses to the pressure you're experiencing.  Many students feel this way during placement season.  You acknowledged you *know* you’re good, which is a significant self-awareness. Let's try to build on that.

One technique from Cognitive Behavioral Therapy (CBT) is to challenge negative thoughts.  When you feel that you're "less than" others, can we try to identify the specific thoughts behind that feeling and then examine if they are truly accurate and helpful?  Often, these thoughts are distortions of reality.